In [32]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

pd.set_option('display.float_format', '{:,.1f}'.format)

In [37]:
def financial_statement(company):
    URL = "https://finance.naver.com/item/main.nhn?code=" + company
    result = requests.get(URL)
    soup = BeautifulSoup(result.content, 'html.parser')
    finance_html = soup.select('div.section.cop_analysis div.sub_section')[0]

    th_data = [item.get_text().strip() for item in finance_html.select('thead th')]
    annual_date = th_data[3:7]
    quarter_date = th_data[7:13]

    finance_index = [item.get_text().strip() for item in finance_html.select('th.h_th2')][3:]
    finance_data = [item.get_text().strip() for item in finance_html.select('td')]

    finance_data = np.array(finance_data)
    finance_data.resize(len(finance_index), 10)

    finance_date = annual_date + quarter_date
    finance = pd.DataFrame(data=finance_data[0:,0:], index=finance_index, columns=finance_date)

    return finance

In [41]:
#대한항공 재무제표

financial_statement('003490')

,2016.12,2017.12,2018.12,2019.12(E),2017.12,2018.03,2018.06,2018.09,2018.12,2019.03(E)
매출액,"117,319","120,922","130,203","133,462","31,071","31,020","31,057","35,179","32,947","32,084"
영업이익,"11,208","9,398","6,403","10,035","2,200","1,663",667,"4,018",54,"2,157"
당기순이익,"-5,568","8,019","-1,857","3,604","3,814",-100,"-3,047","2,573","-1,282",721
영업이익률,9.55,7.77,4.92,7.52,7.08,5.36,2.15,11.42,0.16,6.72
순이익률,-4.75,6.63,-1.43,2.70,12.28,-0.32,-9.81,7.31,-3.89,2.25
ROE(지배주주),-27.24,29.37,-5.90,10.86,29.37,7.17,4.02,9.91,-5.90,
부채비율,"1,178.12",557.10,743.72,,557.10,595.00,654.46,607.77,743.72,
당좌비율,30.26,41.59,37.63,,41.59,47.74,41.16,43.84,37.63,
유보율,110.88,317.41,214.26,,317.41,304.56,238.67,290.89,214.26,
EPS(원),"-7,171","8,631","-2,012","3,700","3,952",-108,"-3,170","2,628","-1,362",95
